# First exploration and growth of Copenhagen bicycle networks
## Project: Bicycle network analysis with Gourab, Sayat, Tyler, Michael, Roberta

This notebook is a first exploration in the project studying the systematic switch from streets to cycling networks in cities.

Contact: Michael Szell (michael.szell@gmail.com)  
Created: 2020-05-26 (Based on Sayat's code)  
Last modified: 2020-05-28

## Preliminaries

### Imports and magics

In [6]:
import networkx as nx
import igraph as ig
import osmnx as ox
import numpy as np
import itertools
from matplotlib import cm
import pandas as pd
from scipy.spatial import distance
from shapely.geometry import Point,MultiPoint,LineString,Polygon
import pandas as pd
import geopandas as gpd

%matplotlib inline
import matplotlib.pyplot as plt

import watermark
%load_ext watermark

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


In [7]:
%watermark -n -v -m -g -iv

osmnx     0.13.0
geopandas 0.7.0
numpy     1.18.4
watermark 2.0.2
pandas    1.0.3
igraph    0.8.2
networkx  2.4
Fri May 29 2020 

CPython 3.8.2
IPython 7.14.0

compiler   : Clang 9.0.1 
system     : Darwin
release    : 19.3.0
machine    : x86_64
processor  : i386
CPU cores  : 12
interpreter: 64bit
Git hash   :


### Functions

In [8]:
def dist(v1,v2):
    dist = distance.euclidean((v1['x'],v1['y']),(v2['x'],v2['y']))
    return dist


def graph_from_plc(graph):
    '''This function takes a open street map object 
    and returns an igraph network with edges weighted 
    with distance between nodes'''
    nodes, edges = ox.graph_to_gdfs(graph)
    g = ig.Graph(directed=False)
    x = nodes['x'].tolist()
    y= nodes['y'].tolist()
    ids = nodes['osmid'].tolist()
    for i in range(len(x)):
        g.add_vertex(x=x[i],y=y[i],id=ids[i])
    df1 = pd.DataFrame(data=g.vs['id'],columns={'id'})
    df1['number'] = np.arange(0,len(list(g.vs())))
    edges1 = pd.merge(df1,edges,left_on='id',right_on='u')
    edges2 = pd.merge(df1,edges1,left_on='id',right_on='v')
    edgelist = (np.stack([edges2['number_x'].tolist(),edges2['number_y'].tolist()]).T).tolist()
    g.add_edges(edgelist)
    g.simplify()
    edge_list = g.get_edgelist()
    weights=[]
    for k in range(len(edge_list)):
        d = dist(g.vs()[edge_list[k][0]],g.vs()[edge_list[k][1]])
        weights.append(d)
    g.es()['weight'] = weights
    
    #bc = g.betweenness(weights='weight')
    return g


def bins_counts(array,a,b,c):
    '''Makes histograms'''
    count, bins = np.histogram(array, bins=np.logspace(a,b,c),density=1)
    t =[]
    for i in range(len(count)):
        t.append((bins[i + 1] + bins[i])/float(2))
    return t,count


def detour (graph):
    '''Calculates the detour factor of a graph'''
    v_x= list(graph.vs()['x'])
    v_y= list(graph.vs()['y'])
    v_x_y = list(zip(v_x, v_y))
    d_g = np.array(graph.shortest_paths_dijkstra(source=list(range(len(v_x_y))),target=list(range(len(v_x_y))),weights='weight')).flatten()
    d_e = scipy.spatial.distance.cdist(v_x_y ,v_x_y ).flatten()
    dist_g=d_g[d_e>0.]
    dist_e=d_e[d_e>0.]
    return(np.mean(np.divide(dist_g,dist_e)))


def closeness(graph):
    '''Finds the node with highest closeness centrality'''
    closeness_values = graph.closeness(weights='weight')
    max_closeness = max(closeness_values)
    index = closeness_values.index(max_closeness)
    return graph.vs(index)['id']

## Exploratory Data Analysis

### Download and prepare bike network of Copenhagen

In [17]:
# This doesnt include Frederiksberg and bridges: ox.graph_from_place("Copenhagen Municipality",
# To do: get shape file with Frederiksberg and bridges

OX_bikeall = ox.graph_from_bbox(55.846724,55.526724,12.730072,12.410072,
                                network_type='bike',
                                retain_all=True,simplify=False)
G_bikeall = graph_from_plc(OX_bikeall)
G_bikeall_lcc = G_bike.subgraph((list(G_bikeall.components()))[0]) # bike network largest connected component


OX_biketrack = ox.graph_from_bbox(55.846724,55.526724,12.730072,12.410072,
                                  network_type='bike',
                                  custom_filter=('["cycleway"~"track"]'),
                                  retain_all=True,simplify=False)
G_biketrack = graph_from_plc(OX_biketrack)


OX_bikelane30 = ox.graph_from_bbox(55.846724,55.526724,12.730072,12.410072,
                                  network_type='bike',
                                  custom_filter=('["cycleway"~"lane"]["maxspeed"~"30"]'),
                                  retain_all=True,simplify=False)
G_bikelane30 = graph_from_plc(OX_bikelane30)


OX_street30 = ox.graph_from_bbox(55.846724,55.526724,12.730072,12.410072,
                                  network_type='drive',
                                  custom_filter=('["maxspeed"~"30"]'),
                                  retain_all=True,simplify=False)
G_street30 = graph_from_plc(OX_street30)

### Cluster preparations

In [34]:
network_id = np.unique(G_biketrack.vs()['id']+G_bikelane30.vs()['id']+G_street30.vs()['id']).tolist() # list of all the node id's in the network
id_dict = dict(zip((G_bikeall_lcc.vs()['id']), np.arange(0,len(list(G_bikeall_lcc.vs()))).tolist())) # dictionary of node id's with consecutive indices

G_agg = G_bikeall_lcc.subgraph([id_dict.get(key) for key in network_id])
# aggregate network of bikeable roads that we want to connect its clusters

# Filtering clusters of size 200 and above
index=[]
for i in range(len(list(G_agg.components()))):
    if len(list(G_agg.components())[i]) > 200:
        index.append(i)


# List of all clusters
clusters=[]
for i in range(len(index)):
    clusters.append(G_agg.subgraph(G_agg.components()[index[i]]))

# Size of clusters
size=[]
for i in range(len(clusters)):
    size.append(len(clusters[i].vs()['id']))

# Indexes of clusters in sorted form (from largest to smallest)
sizeframe= pd.DataFrame(data=size, columns={'cluster_size'})
sizeframe['index'] = np.arange(len(clusters))
sorted_size=sizeframe.sort_values('cluster_size',ascending=False)['index'].tolist()

### Connect clusters

In [ ]:
# Connecting clusters from largest to second largest and so on until all of them are connected
# Each sh_pt is a type of connection
#	below sh_pt connects closest nodes
#		sh_pt_1 connects nodes at random
#	one can add any other type of connection connection

G_1 = clusters[sorted_size[0]].copy()
closest_list=[]
for i in range(1,len(clusters)):
    closest_pair = {'i': 0, 'j': 0, 'dist': np.inf}
    for a in list(G_1.vs()):
        for b in list(clusters[sorted_size[i]].vs()):
            dist_nodes = dist(a,b)
            if dist_nodes < closest_pair['dist']:
                closest_pair['i'] = a['id']
                closest_pair['j'] = b['id']
                closest_pair['dist'] = dist_nodes
    closest_list.append([closest_pair['i'],closest_pair['j']])
    sh_pt = G_bikeall_lcc.get_all_shortest_paths(id_dict.get(closest_pair['i']) ,id_dict.get(closest_pair['j']) ,weights='weight')[0]
    #sh_pt1 = G_bikeall_lcc.get_all_shortest_paths((np.random.choice([id_dict.get(key) for key in G_grw_st_test_r_cl.vs()['id']])),np.random.choice([id_dict.get(key) for key in clusters[sorted_size[i]].vs()['id']]),weights='weight')[0]
    
    nw = np.unique(G_1.vs()['id'] + clusters[sorted_size[i]].vs()['id'] +G_bikeall_lcc.vs(sh_pt)['id'])
    G_1 = G_bikeall_lcc.subgraph([id_dict.get(key) for key in nw])
    print(i,G_1.vcount(),G_1.ecount())


# This is a way to sort nodes according to their betweenness centrality
df1_grw_st= pd.DataFrame(data=G_grw_st.betweenness(weights='weight'),columns={'BC'})
df1_grw_st['x'] = (((G_grw_st.vs())['x']))
df1_grw_st['y'] = (((G_grw_st.vs())['y']))
df1_grw_st['id'] = (((G_grw_st.vs())['id']))
df1_grw_st['no'] = np.arange(0,len(df1_grw_st['x'].tolist())).tolist()
df2_grw_st = df1_grw_st.sort_values('BC',ascending=False)[0:int(len(df1_grw_st['x'])/10)]

## Plotting

In [ ]:
plt.figure(figsize=[4*6.4, 4*4.8])
plt.plot(G_grw_st.vs()['x'],G_grw_st.vs()['y'],'.',color='black')

color=iter(cm.gist_rainbow(np.linspace(0,1,len(clusters))))
for i in range(len(clusters)):
    c=next(color)
    plt.plot(clusters[i].vs['x'],clusters[i].vs['y'],'.',color=c)
#plt.plot(G_grw_st_cl.vs()['x'],G_grw_st_cl.vs()['y'],'.')
plt.axis('off')